# MY Project

let's import the necessary package 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
 
%matplotlib.pyplot inline
from bs4 import BeautifulSoup

Now we will use the request module 

In [13]:
#importing requests package to convert the wiki page to html
import requests
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data=requests.get(url).text

In [18]:
soup=BeautifulSoup(data,'lxml')

In [21]:
#looking for wikitable sortable class since table is in this class
my_table=soup.find('table',{'class':'wikitable sortable'})

In [135]:
import re
link=my_table.findAll(re.compile('td'))
link[1:10]

[<td>Not assigned
 </td>, <td>
 </td>, <td>M2A
 </td>, <td>Not assigned
 </td>, <td>
 </td>, <td>M3A
 </td>, <td>North York
 </td>, <td>Parkwoods
 </td>, <td>M4A
 </td>]

creating a lists of our all three necessary columns 

In [130]:
#this is how our data is stored in link list we need to take care of <td> 
link[0],link[1],link[2]


(<td>M1A
 </td>, <td>Not assigned
 </td>, <td>
 </td>)

In [87]:
postalcode=[]
borough=[]
neighborhood=[]
i=0
while i<len(link):
    a=str(link[i])
    #using string inbuit function to cut the string in between '>' and '\n'
    postalcode.append(a[a.find('>')+1:a.find('\n')])
    a=str(link[i+1])
    borough.append(a[a.find('>')+1:a.find('\n')])
    a=str(link[i+2])
    neighborhood.append(a[a.find('>')+1:a.find('\n')])
    i+=3


Now let's have a look at our Columns
by printing five values

In [97]:
postalcode[1:5],borough[1:5],neighborhood[1:5]

(['M2A', 'M3A', 'M4A', 'M5A'],
 ['Not assigned', 'North York', 'North York', 'Downtown Toronto'],
 ['', 'Parkwoods', 'Victoria Village', 'Regent Park / Harbourfront'])

###### converting to data frame

In [102]:
df=pd.DataFrame()
df['PostalCode']=postalcode
df['Borough']=borough
df['Neighborhood']=neighborhood
df.head(8)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,


Since,the dataframe has been created now we need to refine it according to given instructions

Replace all the cells having Not assigned values in Borough to NaN

In [103]:
#converting Not assigned values to NaN
df['Borough']=df['Borough'].replace('Not assigned',np.nan)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,NaN,
1,M2A,NaN,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Drop all the cells having Nan from df

In [107]:
#dropping rows with NaN values
df.dropna(axis=0,inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Now we will reset the index and replace all '/' with ',' from Neighborhood Column
as per given instructions

In [127]:
for i in range(df.shape[0]):
    df.iloc[i,2]=df.iloc[i,2].replace('/',',')
# reset index
df=df.reset_index(drop=True)
df.head(8)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be converted to same as the borough.

In [128]:
for i in range(df.shape[0]):
    if df.iloc[i,2]=='Not assigned':
        df.iloc[i,2]=df.iloc[i,1]
#print first 10 values
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [129]:
#shape of our dataframe
df.shape

(103, 3)

In [141]:
from geopy.geocoders import Nominatim


In [191]:
dff=pd.read_csv('Geospatial_Coordinates.csv')

In [181]:
#let's have a look at our data set

dff.head(8)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577


since both postal code columns in dff and df dataframe are sorted in different way so let's sort them similarly to make it easy to add Latitude and Longitude

In [183]:
df.sort_values(by='PostalCode',inplace=True)
df=df.reset_index(drop=True)
df.head(8)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park"
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge"


Now let us merge both df and dff dataset but first remove Postal Code column from one on them

In [192]:
#drop attribute 
dff.drop('Postal Code',axis=1,inplace=True)
dff.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


Using join attribute we will join both the dataset

In [186]:
new_data=df.join(dff)

This is how our new_data Data frame looks like

In [189]:
#let's have a look at our dataset
new_data.head(8)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
